In [1]:
import datetime
import os
import pandas as pd
import requests
import smtplib
from bs4 import BeautifulSoup
from email.message import EmailMessage

In [2]:
# Declare Variables
cname = "Assam"
no_of_days = 3

# Date range of past 7 days
dates = [datetime.datetime.now() - datetime.timedelta(days=x) for x in range(0, no_of_days)]
dates = [date.strftime("%d/%m/%Y") for date in dates]
dates = dates[::-1]

sender_email = "techassam@gmail.com"
sender_password = "iyxbvzfejchfpsvt"
recepients = [
    "dodarrang.as@indiapost.gov.in",
    "docachar.as@indiapost.gov.in",
    "donalbari.as@indiapost.gov.in",
    "dotinsukia.as@indiapost.gov.in",
    "doguwahati.as@indiapost.gov.in",
    "dogoalpara.as@indiapost.gov.in",
    "dodibrugarh.as@indiapost.gov.in",
    "dosibsagar.as@indiapost.gov.in",
    "donagaon.as@indiapost.gov.in"
    # 'eliezer.kt@gmail.com',
    # 'tek2991@gmail.com',
]

reports = {
    '3' : 'Details of Bos for which  BO slips are not generated in CSI',
    '23' : 'Details of Post offices where both NSP 1 & 2 are down but ticket is not raised',
    '5' : 'Details of BOs which have not logged in Darpan during Office Hours(9AM - 2PM)'
}

# Check if logs csv exists
if not os.path.exists("logs.csv"):
    df = pd.DataFrame(columns=["date", "report_id", "report_name", "status", "log_date"])
    df.to_csv("logs.csv", index=False)

for date in dates:
    # If date is not provided, use today's date
    if date == "":
        date = datetime.datetime.now().strftime("%d/%m/%Y")

    # url encoding
    rpdate = date.replace("/", "%2f")

    base_url = "https://mis.cept.gov.in/viewDetails.aspx"

    url = base_url + "?cname=" + cname + "&rpdate=" + rpdate
    webpage = requests.get(url)

    # Declare form id
    form_id = "form1"

    # Find form
    form = BeautifulSoup(webpage.text, "html.parser").find("form", {"id": form_id})

    # get the inputs
    inputs = form.find_all("input")

    # create a dictionary of the inputs
    input_dict = {}
    accepted_inputs = ["__VIEWSTATE", "__VIEWSTATEGENERATOR", "__EVENTVALIDATION"]
    for i in inputs:
        if i.has_attr("value") and i["name"] in accepted_inputs:
            input_dict[i["name"]] = i["value"]

    # Create headers
    headers = {
        "authority": "mis.cept.gov.in",
        "method": "POST",
        "path": "/viewDetails.aspx?cname=" + cname + "&rpdate=" + rpdate,
        "scheme": "https"
    }

    # Loop through the reports
    for key, value in reports.items():

        # Check if the report has already been sent
        logs_df = pd.read_csv("logs.csv")
        if logs_df[(logs_df["date"] == date) & (logs_df["report_id"] == int(key))].shape[0] > 0:
            # enter in the logs
            logs_df = logs_df.append({
                "date": date,
                "report_id": key,
                "report_name": value,
                "status": "skipped",
                "log_date": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            }, ignore_index=True)
            logs_df.to_csv("logs.csv", index=False)
            # skip the report
            continue

        # Create a folder by the date inside data folder
        if not os.path.exists("data/" + date.replace("/", "-")):
            os.makedirs("data/" + date.replace("/", "-"))

        # Create a dictionary of the fields
        fields = {
            "ctl00$ContentPlaceHolder1$DdoTransaction": key,
            "ctl00$ContentPlaceHolder1$DdoComplaint" : cname,
            "ctl00$ContentPlaceHolder1$TxtReportDate" : date,
            "ctl00$ContentPlaceHolder1$btnDelete" : "View Data"
        }

        # Combine the inputs and fields
        form_data = {**input_dict, **fields}

        # Send the request
        response = requests.post(url, data=form_data, headers=headers)

        # Get the table
        table = BeautifulSoup(response.text, "html.parser").find("table", {"id": "ContentPlaceHolder1_gvAll"})

        # Check if the table is empty
        if table is None:
            # Create text file with the report name
            with open("data/" + date.replace("/", "-") + "/" + value + "-" + date.replace("/", "-") + ".txt", "w") as f:
                f.write("No Data")
            continue

        # Convert the table to a dataframe
        df = pd.read_html(str(table))[0]

        # Convert the dataframe to a excel file
        df.to_excel("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + ".xlsx", index=False)

        # Field name to group by
        field_name = "DIVISIONNAME"

        # Group by the field name
        grouped = df.groupby(field_name).count()['CIRCLENAME']
        grouped.loc['Total'] = grouped.sum()

        # Rename the column CIRCLENAME to OFFICES
        grouped.rename("OFFICES", inplace=True)

        # Save the grouped data to an excel file
        grouped.to_excel("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + " grouped.xlsx")

        # Prepare subject and body
        subject = value + " " + date.replace("/", "-")
        body = "Sir/Madam,\n\nPlease find the attached report regarding " + value + " for " + date + ".\n\n"

        # Add group data to the body
        body += "Division-wise total:\n"
        for index, row in grouped.iteritems():
            body += index + ": " + str(row) + "\n"

        # Add more text
        body += "\n\nThe report is generated using the MIS portal. Please find the link below:\n"
        body += url + "\n"
        body += "\nPlease find the attached excel file for the report.\n"
        body += "\nIt is hereby requested to kindly download the reports and arrange to submit the same of your division with the reasons at 1100 hours on a daily basis. \n\n"

        # Add signature
        body += "Regards,\n"
        body += "The Asstt. Director (PMU)\n"
        body += "O/o The Chief Postmaster General, Assam Circle\n"
        body += "Meghdoot Bhawan, Guwahati-781001\n"

        # Create email message
        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = subject
        msg['From'] = sender_email
        msg['To'] = recepients

        # Attach the excel file
        with open("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + ".xlsx", "rb") as f:
            file_data = f.read()
            file_name = f.name

        msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)

        # Send the email
        mail_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        mail_server.login(sender_email, sender_password)
        mail_server.set_debuglevel(1)
        mail_server.send_message(msg)
        mail_server.quit()

        # Add the log to the csv
        logs_df = pd.read_csv("logs.csv")
        logs_df = logs_df.append({
            "date": date,
            "report_id": key,
            "report_name": value,
            "status": "sent",
            "log_date": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            }, ignore_index=True)
        logs_df.to_csv("logs.csv", index=False)

send: 'mail FROM:<techassam@gmail.com> size=42940\r\n'
reply: b'250 2.1.0 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp'
send: 'rcpt TO:<dodarrang.as@indiapost.gov.in>\r\n'
reply: b'250 2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp'
send: 'rcpt TO:<docachar.as@indiapost.gov.in>\r\n'
reply: b'250 2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp'
send: 'rcpt TO:<donalbari.as@indiapost.gov.in>\r\n'
reply: b'250 2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.5 OK h10-20020a170902f54a00b0019c13c4b175sm1005708plf.189 - gsmtp'
send: 'rcpt TO:<dotinsukia.as@indiapos